In [1]:
%matplotlib qt
from __future__ import print_function

import rospy
import rosbag
import itertools
from  tqdm import tqdm
from geometry_msgs.msg import PoseWithCovarianceStamped
import tf
import numpy as np

import matplotlib.pyplot as plt
import os

import GPy
import time
from sklearn.cluster import KMeans
import cPickle as pickle

%load_ext autoreload
%autoreload 2

# Train Mag Models

In [ ]:
root_folder = '/mnt/matrix/rosbag/processed_data/nict'
all_maps = 'udai-01'
save_model = False
thr_d = 6


# Load data
print('\n####################\n  map: {:s} \n####################'.format(map_name))

save_filename = '{0:s}/{1:s}/invisible-maps/{1:s}_MagzData.npz'.format(root_folder,map_name)
data = np.load(save_filename,allow_pickle=True)
print(data['XYZ'].shape,data['MZ'].shape)
#plt.plot(data['MZ'])

## Data split
### Too much data to handle using a single GPy

# Save model
save_filename_ = '{0:s}/{1:s}/invisible-maps/{1:s}_MagzModel_'.format(root_folder,map_name)

#create n clusters
print('>> Learning clustering')
n_clusters = 6
kmeans = KMeans(n_clusters=n_clusters).fit(data['XYZ'][:,0:2])

if save_model:
    # save kmeans model
    save_filename = '{:s}kmeans.pkl'.format(save_filename_)
    pickle.dump(kmeans, open(save_filename, "wb"))

c_index = kmeans.predict(data['XYZ'][:,0:2])
print('Cluster centers          npoints')
for i in range(n_clusters):
    print('[{: 9.2f} {: 9.2f}] {: 8d}'.format(kmeans.cluster_centers_[i,0],
                                              kmeans.cluster_centers_[i,1],
                                              int(np.sum(c_index==i))))
print('')
#f, ax = plt.subplots(figsize=(16,9))
#for i in range(n_clusters):
#    index = c_index==i
#    ax.scatter(data['XYZ'][index,0],
#               data['XYZ'][index,1])

print('>> Learning a GPy model per cluster')
for i in range(n_clusters):
    # for points in the cluster
    index = c_index==i
    datax = data['XYZ'][index,:]
    datay = data['MZ'][index,:]

    #f, ax = plt.subplots(figsize=(16,9))
    #ax.scatter(data['XYZ'][:,0],data['XYZ'][:,1],c='gray')
    #ax.scatter(datax[:,0],datax[:,1],c='r')

    # for points not in the cluster (check if thr_d meters from the cluster)
    nindex = c_index!=i
    ndatax = data['XYZ'][nindex,:]
    ndatay = data['MZ'][nindex,:]
    datax_extra = list()
    datay_extra = list()
    for nx,ny in zip(ndatax,ndatay):
        if (np.min(np.linalg.norm(nx-datax,axis=1))) < thr_d:
            datax_extra.append(nx)
            datay_extra.append(ny)
    datax = np.append(datax,np.asarray(datax_extra),axis=0)
    datay = np.append(datay,np.asarray(datay_extra),axis=0)

    print('\n{: 2d} | data points {: 9d}\n'.format(i,datax.shape[0]))

    # train model
    print('  >> Creating model')
    #kernel = GPy.kern.Matern52(3)
    kernel = GPy.kern.RBF(3)
    #kernel.lengthscale[0] = 1.5
    kernel.lengthscale[0] = 1.5
    kernel.lengthscale.fix()
    ##kernel.lengthscale.set_prior(GPy.priors.InverseGamma(1.5,1.5))
    model = GPy.models.GPRegression(datax,datay,kernel=kernel)

    print('  >> Optimizing model')
    t0 = time.time()
    model.optimize()
    print(model)

    print('\n  >> Elapsed min: {:.2f}'.format((time.time()-t0)/60.))

    if save_model:
        save_filename = '{:s}{:02d}.npz'.format(save_filename_,i)
        print('  >> Saving model {:s}'.format(save_filename))
        np.savez_compressed(save_filename,param_array=model.param_array,datax=datax,datay=datay)

    del datax, datax_extra, datay, datay_extra, ndatax, ndatay, kernel, model

# Create Mag Maps

In [ ]:
height = 0.3
res = 0.1

print('\n####################\n  map: {:s} \n####################'.format(map_name))

# Load data
save_filename = '{0:s}/{1:s}/invisible-maps/{1:s}_MagzData.npz'.format(root_folder,map_name)
data = np.load(save_filename,allow_pickle=True)
print(data['XYZ'].shape,data['MZ'].shape)
#plt.plot(data['MZ'])

# create map array
xmin,xmax = np.min(data['XYZ'][:,0]),np.max(data['XYZ'][:,0])
ymin,ymax = np.min(data['XYZ'][:,1]),np.max(data['XYZ'][:,1])
x = np.arange(np.floor(xmin-20),np.ceil(xmax+20)+0.01,step=res)
y = np.arange(np.floor(ymin-20),np.ceil(ymax+20)+0.01,step=res)
print('>> X,Y data: ',x.shape, y.shape)
xy = np.meshgrid(x,y)

XYtest = np.concatenate([xy[0].reshape(-1,1),
                         xy[1].reshape(-1,1),
                         height*np.ones_like(xy[0].reshape(-1,1))],axis=1).astype(np.float32)

save_filename_ = '{0:s}/{1:s}/invisible-maps/{1:s}_MagzModel_'.format(root_folder,map_name)

# load kmeans
print('>> Loading kmeans')
save_filename = '{:s}kmeans.pkl'.format(save_filename_)
kmeans = pickle.load(open(save_filename, "rb"))
## index of XYtest points better predicted by each model
print('>> Computing XY index')
c_index = kmeans.predict(XYtest[:,0:2])

Predictions_mean = np.zeros((XYtest.shape[0],data['MZ'].shape[1]))
Predictions_var = 100*np.ones((XYtest.shape[0],1))

print('>> Computing predictions per model\n'); time.sleep(.2);

for model_i in range(kmeans.n_clusters):
    # load model
    save_filename = '{:s}{:02d}.npz'.format(save_filename_,model_i)
    model_data = np.load(save_filename)
    #kernel = GPy.kern.Matern52(3)
    kernel = GPy.kern.RBF(3)
    #kernel.lengthscale.set_prior(GPy.priors.InverseGamma(1.5,1.5))
    model = GPy.models.GPRegression(model_data['datax'],model_data['datay'],kernel=kernel)
    model[:] = model_data['param_array']
    model.initialize_parameter()

    XYindex = list() 
    #for i,ci in enumerate(c_index):
    #    if ci==model_i: XYindex.append(i)
    ci = c_index==model_i
    XYi = XYtest[ci,:]

    predictions_mean_i = np.zeros((XYi.shape[0],data['MZ'].shape[1]))
    predictions_var_i = 100*np.ones((XYi.shape[0],1))

    # compute predictions in batches
    batch_eval = 10000
    iterations = int(np.ceil(XYi.shape[0]/batch_eval))

    # predictions
    for i in tqdm(range(iterations+1),desc='Model {:02d}/{:02d}'.format(model_i+1,kmeans.n_clusters)):
        try:
            xy = XYi[i*batch_eval:(i+1)*batch_eval,:]
        except:
            xy = XYi[i*batch_eval:-1,:]
        ipoints = xy.shape[0]
        predictions_mean,predictions_variance = model.predict(xy)
        #predictions_mean = model_i*np.ones_like(xy[:,0]).reshape(-1,1)
        #predictions_variance = np.ones_like(xy[:,0]).reshape(-1,1)
        predictions_mean_i[i*batch_eval:i*batch_eval+ipoints,:] = predictions_mean+data['mz_offset']
        predictions_var_i[i*batch_eval:i*batch_eval+ipoints,:]  = predictions_variance

    Predictions_mean[ci] = predictions_mean_i
    Predictions_var[ci] = predictions_var_i

save_filename = '{0:s}/{1:s}/invisible-maps/{1:s}_MagzMap_{2:03d}.npz'.format(root_folder,map_name,int(height*100))
print('  >> Saving model {:s}'.format(save_filename))
np.savez_compressed(save_filename,XY=XYtest,Mag=Predictions_mean,Mag_var=Predictions_var)